In [1]:
import requests
import pickle
from torchvision.transforms.functional import to_tensor,to_pil_image
import gradio as gr
import torch
import torch.nn as nn

c:\Users\danus\miniconda3\envs\lud\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-GPXX0GD8EN/G_trained.pth"

In [3]:
response = requests.get(url)

In [4]:
with open("G_trained.pth", "wb") as f:
    f.write(response.content)

In [5]:
latent_vector_size = 128

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_vector_size,64*8,4,1,0,bias = False),
            nn.BatchNorm2d(64*8),
            nn.ReLU(True),
            nn.ConvTranspose2d(64*8,64*4,4,2,1,bias = False),
            nn.BatchNorm2d(64*4),
            nn.ReLU(True),
            nn.ConvTranspose2d(64*4,64*2,4,2,1,bias = False),
            nn.BatchNorm2d(64*2),
            nn.ReLU(True),
            nn.ConvTranspose2d(64*2,64,4,2,1,bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,3,4,2,1,bias = False),
            nn.Tanh()
        )
    def forward(self,input):
        output = self.main(input)
        return output

In [7]:
G = Generator()

In [8]:
device = torch.device('cpu')
G.load_state_dict(torch.load('G_trained.pth',map_location = device))

<All keys matched successfully>

In [10]:

def make_image(a, b, value):
    shape =1
    z = a * torch.randn(shape, latent_vector_size,1,1) + b
    xhat = G(z)[0].detach().squeeze(0)
    
    xhat = (xhat - xhat.min())/(xhat.max() - xhat.min())
    
    #resize the image 
    
    image = to_pil_image(xhat)
    width,height = image.size 
    fixed_size = 1000
    new_width = fixed_size
    resized_image = iage.resize((new_width, int(new_width * height / width)))
    return resized_image
    

In [11]:
title = "Anime Creation App"
css = ".output_image {height: 60rem !important; width: 100% !important;}"

In [20]:
# Create a Gradio Interface
gr.Interface(
    fn=make_image,
    inputs=[
        gr.Slider(1, 10, label='Variation', value=1),
        gr.Slider(-5, 5, label='Bias', value=0),
        gr.Slider(-5, 5, label="Fine Tune : LAtent VariableValue", value=0)
    ],
    title=title,
    css=css,
    outputs=gr.Image(elem_id="output_image")
).launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
